In [87]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report
from imblearn.over_sampling import RandomOverSampler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

#  Import and read the charity_data.csv.
import pandas as pd 
stroke_data_df = pd.read_csv("healthcare-dataset-stroke-data.csv")
stroke_data_df.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Preprocessing**

In [16]:
# Drop the non-beneficial ID column.
stroke_df = stroke_data_df.drop(columns={'id'})

In [17]:
# Drop rows containing NaN
stroke_df = stroke_df.dropna()

In [18]:
# Convert categorical data to numeric with `pd.get_dummies`
encoded_stroke_data = pd.get_dummies(stroke_df)
encoded_stroke_data

,age,hypertension,heart_disease,avg_glucose_level,bmi,stroke,gender_Female,gender_Male,gender_Other,ever_married_No,...,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,Residence_type_Rural,Residence_type_Urban,smoking_status_Unknown,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
0,67.0,0,1,228.69,36.6,1,0,1,0,0,...,0,1,0,0,0,1,0,1,0,0
2,80.0,0,1,105.92,32.5,1,0,1,0,0,...,0,1,0,0,1,0,0,0,1,0
3,49.0,0,0,171.23,34.4,1,1,0,0,0,...,0,1,0,0,0,1,0,0,0,1
4,79.0,1,0,174.12,24.0,1,1,0,0,0,...,0,0,1,0,1,0,0,0,1,0
5,81.0,0,0,186.21,29.0,1,0,1,0,0,...,0,1,0,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5104,13.0,0,0,103.08,18.6,0,1,0,0,1,...,0,0,0,1,1,0,1,0,0,0
5106,81.0,0,0,125.20,40.0,0,1,0,0,0,...,0,0,1,0,0,1,0,0,1,0
5107,35.0,0,0,82.99,30.6,0,1,0,0,0,...,0,0,1,0,1,0,0,0,1,0
5108,51.0,0,0,166.29,25.6,0,0,1,0,0,...,0,1,0,0,1,0,0,1,0,0


In [19]:
# Split our preprocessed data into our features and target arrays
y = encoded_stroke_data["stroke"]
X = encoded_stroke_data.drop(["stroke"], axis=1)

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [20]:
# Check the balance of our target values
y.value_counts()

0    4700
1     209
Name: stroke, dtype: int64

In [21]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# **Logistic Regression Model**
Attempt 1 using orignial data.

Something is clearly going wrong here, but I'm not sure what.

In [34]:
# Fit the model
# Instantiate the Logistic Regression model
logistic_regression_model = LogisticRegression(solver='lbfgs', max_iter=200)

# Fit the model using training data
model = logistic_regression_model.fit(X_train_scaled, y_train)

In [35]:
# Make a prediction using the testing data
LR_pred = logistic_regression_model.predict(X_test_scaled)

In [36]:
# Print the balanced_accuracy score of the model
balanced_accuracy_score(y_test, LR_pred)

0.5

In [37]:
# Generate a confusion matrix for the model
matrix = confusion_matrix(y_test, LR_pred)
print(matrix)

[[1184    0]
 [  44    0]]


In [38]:
# Print the classification report for the model
report = classification_report(y_test, LR_pred)
print(report)

              precision    recall  f1-score   support

           0       0.96      1.00      0.98      1184
           1       0.00      0.00      0.00        44

    accuracy                           0.96      1228
   macro avg       0.48      0.50      0.49      1228
weighted avg       0.93      0.96      0.95      1228



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# **Neural Network Model**
Attempt 1 using original data.

In [27]:
# Define the model
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 9
hidden_nodes_layer2 = 9
hidden_nodes_layer3 = 9

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 9)                 198       
                                                                 
 dense_1 (Dense)             (None, 9)                 90        
                                                                 
 dense_2 (Dense)             (None, 9)                 90        
                                                                 
 dense_3 (Dense)             (None, 1)                 10        
                                                                 
Total params: 388
Trainable params: 388
Non-trainable params: 0
_________________________________________________________________


In [28]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [29]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=50)

Epoch 1/50
116/116 [==============================] - 2s 3ms/step - loss: 0.4255 - accuracy: 0.9329
Epoch 2/50
116/116 [==============================] - 0s 3ms/step - loss: 0.2101 - accuracy: 0.9552
Epoch 3/50
116/116 [==============================] - 0s 3ms/step - loss: 0.1767 - accuracy: 0.9552
Epoch 4/50
116/116 [==============================] - 0s 3ms/step - loss: 0.1661 - accuracy: 0.9552
Epoch 5/50
116/116 [==============================] - 0s 3ms/step - loss: 0.1600 - accuracy: 0.9552
Epoch 6/50
116/116 [==============================] - 0s 3ms/step - loss: 0.1560 - accuracy: 0.9552
Epoch 7/50
116/116 [==============================] - 0s 3ms/step - loss: 0.1541 - accuracy: 0.9552
Epoch 8/50
116/116 [==============================] - 0s 3ms/step - loss: 0.1523 - accuracy: 0.9552
Epoch 9/50
116/116 [==============================] - 0s 3ms/step - loss: 0.1504 - accuracy: 0.9552
Epoch 10/50
116/116 [==============================] - 0s 3ms/step - loss: 0.1494 - accuracy: 0.9552

In [30]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

39/39 - 0s - loss: 0.1273 - accuracy: 0.9634 - 306ms/epoch - 8ms/step
Loss: 0.12730517983436584, Accuracy: 0.9633550643920898


In [ ]:
# Export our model to HDF5 file
# nn.save('Models/stroke_model_NN_1.h5')

# **K Nearest Neighbors**

In [66]:
# Instantiate the model with k = 3 neighbors
model = KNeighborsClassifier(n_neighbors=3)

In [67]:
# Train the model
model.fit(X_train_scaled, y_train)

KNeighborsClassifier(n_neighbors=3)

In [69]:
# Create predictions
KN_pred = model.predict(X_test_scaled)

In [70]:
# Print confusion matrix
confusion_matrix(KN_pred,y_test)

array([[1177,   43],
       [   7,    1]])

In [71]:
# Print classification report
print(classification_report(KN_pred,y_test))

              precision    recall  f1-score   support

           0       0.99      0.96      0.98      1220
           1       0.02      0.12      0.04         8

    accuracy                           0.96      1228
   macro avg       0.51      0.54      0.51      1228
weighted avg       0.99      0.96      0.97      1228



# **Random Forest**

It looks like this is having the same problem as the first Logistic Regression model, using the resampled data might help.

In [88]:
# Create a random forest classifier
rf_model = RandomForestClassifier(n_estimators=500)

In [89]:
# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

In [90]:
# Making predictions using the testing data
rf_pred = rf_model.predict(X_test_scaled)

In [91]:
# Print confusion matrix
confusion_matrix(rf_pred,y_test)

array([[1184,   44],
       [   0,    0]])

In [92]:
# Print classification report
print(classification_report(rf_pred,y_test))

              precision    recall  f1-score   support

           0       1.00      0.96      0.98      1228
           1       0.00      0.00      0.00         0

    accuracy                           0.96      1228
   macro avg       0.50      0.48      0.49      1228
weighted avg       1.00      0.96      0.98      1228



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# **Resample Data with RandomOverSampler**

In [31]:
!pip install imblearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [45]:

# Instantiate the random oversampler model
ros = RandomOverSampler()

# Fit the original training data to the random_oversampler model
X_R, y_R = ros.fit_resample(X_train, y_train)

In [46]:
# Count the distinct values of the resampled labels data
y_R.value_counts()

0    3516
1    3516
Name: stroke, dtype: int64

In [47]:
X_train_scaled_R = X_scaler.transform(X_R)

# **Neural Network Model 2**
Attempt 2 using RandomOverSampler

In [73]:
# Neural network model with RandomOverSampler
# Define the model
number_input_features = len(X_train_scaled_R[0])
hidden_nodes_layer1 = 9
hidden_nodes_layer2 = 9
hidden_nodes_layer3 = 9

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 9)                 198       
                                                                 
 dense_9 (Dense)             (None, 9)                 90        
                                                                 
 dense_10 (Dense)            (None, 9)                 90        
                                                                 
 dense_11 (Dense)            (None, 1)                 10        
                                                                 
Total params: 388
Trainable params: 388
Non-trainable params: 0
_________________________________________________________________


In [74]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [75]:
# Train the model
fit_model = nn.fit(X_train_scaled_R,y_R,epochs=50)

Epoch 1/50
220/220 [==============================] - 1s 2ms/step - loss: 0.6131 - accuracy: 0.6638
Epoch 2/50
220/220 [==============================] - 0s 2ms/step - loss: 0.5059 - accuracy: 0.7622
Epoch 3/50
220/220 [==============================] - 0s 2ms/step - loss: 0.4747 - accuracy: 0.7749
Epoch 4/50
220/220 [==============================] - 0s 2ms/step - loss: 0.4600 - accuracy: 0.7860
Epoch 5/50
220/220 [==============================] - 0s 2ms/step - loss: 0.4486 - accuracy: 0.7944
Epoch 6/50
220/220 [==============================] - 0s 2ms/step - loss: 0.4388 - accuracy: 0.8028
Epoch 7/50
220/220 [==============================] - 0s 2ms/step - loss: 0.4295 - accuracy: 0.8046
Epoch 8/50
220/220 [==============================] - 0s 2ms/step - loss: 0.4195 - accuracy: 0.8084
Epoch 9/50
220/220 [==============================] - 0s 2ms/step - loss: 0.4111 - accuracy: 0.8094
Epoch 10/50
220/220 [==============================] - 0s 2ms/step - loss: 0.4047 - accuracy: 0.8150

In [76]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

39/39 - 0s - loss: 0.4411 - accuracy: 0.8176 - 195ms/epoch - 5ms/step
Loss: 0.44111236929893494, Accuracy: 0.8175895810127258


In [ ]:
# Export our model to HDF5 file
# nn.save('Models/stroke_model_NN_2.h5')

# **Logistic Regression Model 2**
Attempt 2 using RandomOverSampler

In [53]:
# Logistic Regression model with RandomOverSampler
# Instantiate the Logistic Regression model
ros_logistic_regression_model = LogisticRegression(solver='lbfgs', max_iter=200)

# Fit the model using the resampled training data
ros_model = logistic_regression_model.fit(X_train_scaled_R, y_R)

# Make a prediction using the testing data
ros_LR_pred = logistic_regression_model.predict(X_test_scaled)

In [54]:
# Print the balanced_accuracy score of the model 
balanced_accuracy_score(y_test, ros_LR_pred)

0.8105804668304668

In [55]:
# Generate a confusion matrix for the model
ros_matrix = confusion_matrix(y_test, ros_LR_pred)
print(ros_matrix)

[[870 314]
 [  5  39]]


In [56]:
# Print the classification report for the model
ros_report = classification_report(y_test, ros_LR_pred)
print(ros_report)

              precision    recall  f1-score   support

           0       0.99      0.73      0.85      1184
           1       0.11      0.89      0.20        44

    accuracy                           0.74      1228
   macro avg       0.55      0.81      0.52      1228
weighted avg       0.96      0.74      0.82      1228



# **Neural Network Model 2**
Attempt 3 using Keras Tuner and RandomOverSampler

In [57]:
!pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 10.3 MB/s eta 0:00:00


In [58]:
import keras_tuner as kt

In [82]:
# Set value for input_dim
number_input_features = len(X_train_scaled_R[0])

# Create method for new Sequential model with hyperparameter option
def create_model (hp):
  nn_model = tf.keras.models.Sequential()

  # Activation function options
  activation = hp.Choice('activation', ['relu', 'tanh', 'sigmoid'])

  # Number of neurons in first layer
  nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units', min_value=1, max_value=20, step=4),
                                     activation=activation, input_dim=number_input_features))
  
  # Number of hidden layers and neurons per layer
  for i in range(hp.Int('num_layers', 1, 6)):
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i), min_value=1, max_value=20, step=4), activation=activation))

  nn_model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

  # Compile the model
  nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

  return nn_model


In [83]:
# Create tuner with keras_tuner
tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2)

In [84]:
# Run tuner search for best hyperparameters
tuner.search(X_train_scaled_R,y_R,epochs=20,validation_data=(X_test_scaled,y_test))

In [85]:
# Find most accurate model
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'tanh',
 'first_units': 17,
 'num_layers': 6,
 'units_0': 13,
 'units_1': 17,
 'units_2': 5,
 'units_3': 5,
 'units_4': 1,
 'units_5': 13,
 'tuner/epochs': 20,
 'tuner/initial_epoch': 7,
 'tuner/bracket': 1,
 'tuner/round': 1,
 'tuner/trial_id': '0048'}

In [81]:
# Evaluate best model against full test data
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

39/39 - 0s - loss: 0.1325 - accuracy: 0.9650 - 329ms/epoch - 8ms/step
Loss: 0.1324974149465561, Accuracy: 0.9649837017059326
